In [3]:
import pandas as pd
import os
datasets = ['amber', 'pcmsolver', 'hoomd', 'rmg-py', 'xenon']

In [3]:
acc = {}
df = pd.ExcelFile("amber_sample_1.xlsx").parse("amber_sample")
print(df.shape)
print(df.columns)

(100, 3)
Index([u'Document Title', u'Abstract', u'label'], dtype='object')


In [47]:
for d in datasets:
    acc[d] = {0: [0, 0], 1: [0, 0]}
    fr_label = pd.read_csv(d + "_sample.csv")['label'].values.tolist()
    sample_file_1 = pd.ExcelFile(d + "_sample_1.xlsx").parse(d + "_sample")
    sample_file_2 = pd.ExcelFile(d + "_sample_2.xlsx").parse(d + "_sample")
    label_sample_1 = sample_file_1['label'].values.tolist()
    label_sample_2 = sample_file_2['label'].values.tolist()
    length = len(fr_label)
    for i in range(length):
        acc[d][fr_label[i]][1] += 1
        if math.isnan(label_sample_1[i]):
            label_sample_1[i] = 0
        if math.isnan(label_sample_2[i]):
            label_sample_2[i] = 0
        if fr_label[i] == label_sample_1[i] and fr_label[i] == label_sample_2[i]:
            acc[d][fr_label[i]][0] += 1

In [48]:
for d in acc.keys():
    print(acc[d])
    print(d, float(acc[d][0][0])/acc[d][0][1], float(acc[d][1][0])/acc[d][1][1])
    print(float(acc[d][0][0] + acc[d][1][0])/(acc[d][0][1]+acc[d][1][1]))
    print()

{0: [44, 55], 1: [39, 45]}
('amber', 0.8, 0.8666666666666667)
0.83
()
{0: [1, 16], 1: [56, 84]}
('hoomd', 0.0625, 0.6666666666666666)
0.57
()
{0: [71, 75], 1: [20, 25]}
('rmg-py', 0.9466666666666667, 0.8)
0.91
()
{0: [25, 42], 1: [35, 58]}
('pcmsolver', 0.5952380952380952, 0.603448275862069)
0.6
()
{0: [17, 40], 1: [24, 60]}
('xenon', 0.425, 0.4)
0.41
()


In [5]:
from collections import Counter
import math
majority_voted_labels = {}
for d in datasets:
    majority_voted_labels[d] = []   
    fr_label = pd.read_csv(d + "_sample.csv")['label'].values.tolist()
    sample_file_1 = pd.ExcelFile(d + "_sample_1.xlsx").parse(d + "_sample")
    sample_file_2 = pd.ExcelFile(d + "_sample_2.xlsx").parse(d + "_sample")
    label_sample_1 = sample_file_1['label'].values.tolist()
    label_sample_2 = sample_file_2['label'].values.tolist()
    length = len(fr_label)
    for i in range(length):
        if math.isnan(label_sample_1[i]):
            label_sample_1[i] = 0
        l = [fr_label[i], label_sample_1[i], label_sample_2[i]]
        most_common, num_most_common = Counter(l).most_common(1)[0]
        
        #major_l_i = max(set(l), key = l.count)
        majority_voted_labels[d].append(most_common)

In [43]:
from sklearn.metrics import confusion_matrix
for d in datasets:
    fr_label = pd.read_csv(d + "_sample.csv")['label'].values.tolist()
    #print(fr_label)
    #print(majority_voted_labels[d])
    tn, fp, fn, tp = confusion_matrix(majority_voted_labels[d], fr_label).ravel()
    print(d, tn, fp, fn, tp)

('amber', 46, 2, 9, 43)
('pcmsolver', 33, 16, 9, 42)
('hoomd', 14, 2, 2, 82)
('rmg-py', 72, 2, 3, 23)
('xenon', 38, 10, 2, 50)


In [ ]:
from collections import Counter
import math
majority_voted_labels = {}
for d in datasets:
    majority_voted_labels[d] = []   
    fr_label = pd.read_csv(d + "_sample.csv")['label'].values.tolist()
    length = len(fr_label)
    for i in range(length):
        if math.isnan(label_sample_1[i]):
            label_sample_1[i] = 0
        l = [fr_label[i], label_sample_1[i], label_sample_2[i]]
        most_common, num_most_common = Counter(l).most_common(1)[0]
        
        #major_l_i = max(set(l), key = l.count)
        majority_voted_labels[d].append(most_common)

In [9]:
from nltk.stem import PorterStemmer
from nltk.tokenize import sent_tokenize, word_tokenize
import string

ps = PorterStemmer()

for d in datasets:
    df = pd.read_csv(d + "_sample.csv")
    commits_m = df['Abstract'].values.tolist()
    buggy_commits = [0]*len(commits_m)
    index = 0
    check = set()
    for m in commits_m:
        try:
            words = word_tokenize(m)
            stemmed_w = [ps.stem(w) for w in words]
        except TypeError:
            print("TypeError", m)
            break
        except UnicodeDecodeError:
            print("UDError", words)
            break
        #print(stemmed_w)
        bug_key_words = set(["bug", "fix", "wrong", "error", "fail", "problem", "patch"])
        check = set(stemmed_w).intersection(bug_key_words)
        if len(check) > 0:
            buggy_commits[index] = 1
        index += 1
    df["baseline"] = buggy_commits
        #final_sw = []
        #flag = False
        #for w in stemmed_w:
            #if len(w) > 2 or w[0] in (string.digits + string.letters):
                #final_sw.append(w)
                #if not flag and w in ["bug", "fix", "wrong", "error", "fail", "problem", "patch"]:
                    #flag = True
                    #buggy_commits.append(index)
    

In [13]:
from sklearn.metrics import confusion_matrix

for d in datasets:
    fr_label = df["baseline"].values.tolist()
    #print(fr_label)
    #print(majority_voted_labels[d])
    tn, fp, fn, tp = confusion_matrix(majority_voted_labels[d], fr_label).ravel()
    print(d, tn, fp, fn, tp)


('amber', 19, 29, 23, 29)
('pcmsolver', 21, 28, 21, 30)
('hoomd', 4, 12, 38, 46)
('rmg-py', 31, 43, 11, 15)
('xenon', 39, 9, 3, 49)
